# Import Libraries

In [ ]:
# for creating neural network architecture
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

# for image processing
import cv2
import numpy as np

# for dataset management
import os, shutil

# for time management
from tqdm import tqdm

# Setting up hardware

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')

if physical_devices != []:
    print("Using GPU")
    for i in physical_devices:
        tf.config.experimental.set_memory_growth(i, True)
else:
    print("Using CPU")
    pass

# Load Dataset

In [ ]:
root_dir = str(input("Path where 'classify train' directory belongs: "))
classify_train = os.path.join(root_dir, 'classify train')

train_directory = os.path.join(classify_train, 'training')
validation_directory = os.path.join(classify_train, 'validation')
test_directory = os.path.join(classify_train, 'testing')

# Callbacks

In [ ]:
learning_rate = 0.001
steps = 1 # change steps to 1 to apply exponential decay

def lr_schedule(epoch):
    return learning_rate * (0.1 ** int(epoch / steps))


callback = [tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose = 1),
            tf.keras.callbacks.EarlyStopping(monitor = 'loss', min_delta = 0.001, patience = 10, verbose = 1, mode = "min")]

# Generator

In [ ]:
class_no = len(os.listdir(train_directory))

print("This is a " + str(class_no) + "-Class Classification")

if class_no <= 2:
    class_mode = 'binary'
    output_activation = 'sigmoid'
    output_neurons = 1
    losses = 'binary_crossentropy'

else:
    class_mode = 'categorical'
    output_activation = 'softmax'
    output_neurons = class_no
    losses = 'categorical_crossentropy'

In [ ]:
h = int(input("Image Dimension(H or W): "))
w = h
target_size = (h,w)
color = int(input("Press 1 for RGB \nPress 2 for Grayscale "))
if color == 1:
    color_mode = 'rgb'
    dim = (h,w,3)
elif color == 2:
    color_mode = 'grayscale'
    dim = (h,w,1)

# Model Architecture

**Don't put user inputs inside the function below as it'll be called multiple times inside a loop**

In [ ]:
def MobileNet():
    print("\nTRAINING ON MobileNet MODEL:-")

    full_model = tf.keras.applications.MobileNet(input_shape = (224,224,3), weights = 'imagenet', include_top = True)
    full_model.summary()
    
    no_d_layers = 1
    d_neurons = 128
    d_dropout = 0.2
    
    base_model = tf.keras.applications.MobileNet(input_shape = dim, weights = 'imagenet', include_top = False)

    x = base_model.output
    
    x = GlobalMaxPooling2D()(x)
    
    m, n = 0, 0
    for d in range(no_d_layers):
        m = 2**d
        n = d_neurons//m
        x = Dense(n, kernel_regularizer=l2(lambd), bias_regularizer=l2(lambd))(x)
        x = LeakyReLU()(x)
        x = Dropout(d_dropout)(x)
        x = BatchNormalization()(x)
    
    predictions = Dense(output_neurons, activation = output_activation)(x)  

    model = Model(inputs = base_model.input, outputs=predictions)

    train_base_model = "Y"
    if train_base_model.upper() == 'Y':
        for layer in base_model.layers:
            layer.trainable = True
    elif train_base_model.upper() == 'N':
        for layer in base_model.layers:
            layer.trainable = False

    return model

# Research

In [ ]:
epoch = 100

lambd_list = []
lambd_no = int(input('Using how many lambdas you want to train with ? '))
print('Enter {} lambda value/values consecutively:'.format(lambd_no))
for i in range(lambd_no): 
    lambd = float(input('Enter lambda value: '))
    lambd_list.append(lambd)

batch_list = []
batch_no = int(input("Enter how many batches you want to use: "))
print('Enter {} batch no. consecutively:'.format(batch_no))
for j in range(batch_no):
    b_size = int(input('Enter batch size: '))
    batch_list.append(b_size)

In [ ]:
# iterate over batch sizes
for batch_size in batch_list: 
    train_datagen = ImageDataGenerator(rescale=1.0/255.0)
    train_generator = train_datagen.flow_from_directory(train_directory,
                                                        batch_size = batch_size,
                                                        class_mode = class_mode,
                                                        target_size = target_size)

    val_datagen = ImageDataGenerator(rescale=1.0/255.0)
    validation_generator = val_datagen.flow_from_directory(validation_directory,
                                                        batch_size = batch_size,
                                                        class_mode = class_mode,
                                                        target_size = target_size)

    test_datagen = ImageDataGenerator(rescale=1.0/255.0)
    test_generator = test_datagen.flow_from_directory(test_directory,
                                                        batch_size = batch_size,
                                                        class_mode = class_mode,
                                                        target_size = target_size)
    
    # iterate over lambdas
    for l in lambd_list:
        model = MobileNet()
        
        # COMPILE
        loss = 'categorical_crossentropy'
        optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)
        model.compile(loss = loss,
                    optimizer = optimizer,
                    metrics=['accuracy',
                    tf.keras.metrics.TruePositives(), 
                    tf.keras.metrics.TrueNegatives(), 
                    tf.keras.metrics.FalsePositives(),
                    tf.keras.metrics.FalseNegatives()])

        print ('\n************ for lambda = {}************\n'.format(lambd))

        # FIT
        history = model.fit(train_generator,
                        epochs = epoch,
                        verbose = 1,
                        callbacks = callback,
                        validation_data = validation_generator,
                        shuffle = True)

        # PLOT
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        epochs = range(len(acc))
        import matplotlib.pyplot as plt
        
        # Accuracy vs Epochs
        plt.plot(epochs, acc, 'r', label='Training Accuracy')
        plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")
        plt.title('Training and validation accuracy vs Epochs')
        plt.legend()
        plt.show()
        
        # Loss vs Epochs
        plt.plot(epochs, loss, 'r', label="Training Loss")
        plt.plot(epochs, val_loss, 'b', label="Validation Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.title('Training and validation loss vs Epochs')
        plt.legend()
        plt.show()

        # ACCURACIES
        print("Training accuracy: {}".format(model.evaluate(train_generator, verbose=0)[1]))
        print("Validation accuracy: {}".format(model.evaluate(validation_generator, verbose=0)[1]))
        print("Blind test accuracy: {}".format(model.evaluate(test_generator, verbose=0)[1]))
        tp = int(model.evaluate(test_generator, verbose=0)[2])
        tn = int(model.evaluate(test_generator, verbose=0)[3])
        fp = int(model.evaluate(test_generator, verbose=0)[4])
        fn = int(model.evaluate(test_generator, verbose=0)[5])
        sensitivity = (tp/(tp+fn))*100
        specificity = (tn/(tn+fp))*100
        print("Sensitivity: {}".format(sensitivity))
        print("Specificity: {}".format(specificity))